## project 9727 
### Wenhao Chen 
### ZID:z5442760
### part 1

In [2]:
# Loading the Data
import pandas as pd

# 将 CSV 文件导入为 DataFrame
spotify_df = pd.read_csv('spotify_dataset.csv',on_bad_lines='skip', quotechar='"')
song_df = pd.read_csv("spotify_millsongdata.csv",on_bad_lines='skip', quotechar='"')

spotify_df = spotify_df.dropna()
song_df = song_df.dropna()

# 打印前几行数据以确认导入成功
print(spotify_df.head())
print(song_df.head())

                            user_id                      "artistname"  \
0  9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   
1  9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
2  9cc0cfd4d7d7885102480dd99e7a90d6                      Tiffany Page   
3  9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
4  9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   

                                         "trackname"  "playlistname"  
0               (The Angels Wanna Wear My) Red Shoes  HARD ROCK 2010  
1  (What's So Funny 'Bout) Peace, Love And Unders...  HARD ROCK 2010  
2                                   7 Years Too Late  HARD ROCK 2010  
3                              Accidents Will Happen  HARD ROCK 2010  
4                                             Alison  HARD ROCK 2010  
  artist                   song                                        link  \
0   ABBA  Ahe's My Kind Of Girl  /a/abba/ahes+my+kind+of

In [3]:
spotify_df.columns = spotify_df.columns.str.replace('"', '').str.strip()

print(spotify_df.info())
print(song_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 12856831 entries, 0 to 12891679
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   user_id       object
 1   artistname    object
 2   trackname     object
 3   playlistname  object
dtypes: object(4)
memory usage: 490.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB
None


In [4]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Define preprocessing function

def preprocess_text(text):
    if pd.isnull(text):
        return ""
    if not isinstance(text, str):
        return str(text)
    text = text.lower()
    text = re.sub(r'[^\w\s()-]', '', text)   #######################
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens] ####################
    return ' '.join(tokens) 

spotify_df['playlistname'] = spotify_df['playlistname'].apply(preprocess_text)
# spotify_df = spotify_df.applymap(preprocess_text)
# song_df = spotify_df.applymap(preprocess_text)

print(spotify_df.head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\forsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\forsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\forsa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                            user_id                        artistname  \
0  9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   
1  9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
2  9cc0cfd4d7d7885102480dd99e7a90d6                      Tiffany Page   
3  9cc0cfd4d7d7885102480dd99e7a90d6  Elvis Costello & The Attractions   
4  9cc0cfd4d7d7885102480dd99e7a90d6                    Elvis Costello   

                                           trackname    playlistname  
0               (The Angels Wanna Wear My) Red Shoes  hard rock 2010  
1  (What's So Funny 'Bout) Peace, Love And Unders...  hard rock 2010  
2                                   7 Years Too Late  hard rock 2010  
3                              Accidents Will Happen  hard rock 2010  
4                                             Alison  hard rock 2010  


In [10]:
import random

# 获取随机10个用户的歌单列表
random_users = random.sample(list(spotify_df['user_id'].unique()), 10)
user_playlists = spotify_df[spotify_df['user_id'].isin(random_users)]

# 从用户歌单中随机提取10个不重复的歌单
unique_playlists = user_playlists[['playlistname']].drop_duplicates()
random_playlists = unique_playlists.sample(n=10, replace=False)
print("\n随机提取的10个不重复歌单 作为随机推荐:")
print(random_playlists)

# 基于关键词进行推荐
def recommend_playlists(keyword, df, top_n=10):
    keyword = preprocess_text(keyword)
    df['keyword_match'] = df['playlistname'].apply(lambda x: keyword in x)
    recommendations = df[df['keyword_match']].drop_duplicates(subset=['playlistname']).head(top_n)
    return recommendations['playlistname'].tolist()

# 示例关键词
keyword = "love"

# 推荐10条相关的 playlistname
recommendations = recommend_playlists(keyword, spotify_df)

print("\n基于关键词 '{}' 的推荐歌单:".format(keyword))
for i, playlist in enumerate(recommendations, 1):
    print("{}. {}".format(i, playlist))



随机提取的10个不重复歌单 作为随机推荐:
                                  playlistname
7773955                                 u loud
131568                            mount kimble
10668163             sun kil moon - among leaf
944673                                nocturne
944469                                 chicago
944496                             de beauvoir
10667114  lazify - james brown - funky drummer
10667139        lazify - killing joke - eighty
132823                                trip-hop
683831                               barricada

基于关键词 'love' 的推荐歌单:
1. enrique iglesias sex love ( deluxe )
2. gave love
3. john legend - love future
4. john newman - love
5. yelawolf love story
6. love jaïpur
7. love ( feat josh jakq )
8. last chance love
9. lastfm loved track
10. derek domino layla assorted love song


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics.pairwise import cosine_similarity

def extract_keywords(text):
    vectorizer = CountVectorizer(max_features=10)
    X = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out()

# 为每个歌单获取10个关键词
playlist_keywords = {}
playlist_tracknames = {}
for playlist in random_playlists['playlistname']:
    tracknames = ' '.join(user_playlists[user_playlists['playlistname'] == playlist]['trackname'].tolist())
    keywords = extract_keywords(tracknames)
    playlist_keywords[playlist] = keywords
    playlist_tracknames[playlist] = tracknames

print("\n每个歌单的关键词:")
for playlist, keywords in playlist_keywords.items():
    print(f"{playlist}: {', '.join(keywords)}")

# 准备数据用于训练贝叶斯分类器
data = []
labels = []
for playlist, keywords in playlist_keywords.items():
    for keyword in keywords:
        data.append(keyword)
        labels.append(playlist)

# 使用 CountVectorizer 将关键词转换为特征向量
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)
y = labels

# 使用 Multinomial Naive Bayes 训练模型
mnb = MultinomialNB()
mnb.fit(X, y)

# 基于输入关键词推荐新的歌单
def recommend_playlist_based_on_keyword(keyword, model, vectorizer, tracknames_dict, top_n=10):
    keyword_processed = preprocess_text(keyword)
    X_new = vectorizer.transform([keyword_processed])
    predicted_playlist = model.predict(X_new)[0]

    # 计算相似度
    input_vector = vectorizer.transform([keyword_processed])
    similarities = []
    for playlist, tracknames in tracknames_dict.items():
        playlist_vector = vectorizer.transform([tracknames])
        similarity = cosine_similarity(input_vector, playlist_vector)
        similarities.append((playlist, similarity[0][0]))

    # 根据相似度排序并推荐
    sorted_playlists = sorted(similarities, key=lambda x: x[1], reverse=True)
    recommended_playlists = [playlist for playlist, _ in sorted_playlists[:top_n]]
    return recommended_playlists

# 示例关键词
input_keyword = "love"

# 推荐基于输入关键词的新歌单
recommended_playlists = recommend_playlist_based_on_keyword(input_keyword, mnb, vectorizer, playlist_tracknames)
print(f"\n基于关键词 '{input_keyword}' 的推荐歌单:")
for i, playlist in enumerate(recommended_playlists, 1):
    print(f"{i}. {playlist}")


每个歌单的关键词:
u loud: for, girl, in, it, little, love, night, one, the, you
mount kimble: feat, king, krule, many, so, stray, sullen, that, time, times
sun kil moon - among leaf: attractive, blues, not, talented, that, the, uk, was, yet, young
nocturne: 39, act, flat, in, la, no, nocturne, of, op, parsifal
chicago: and, east, me, news, of, on, part, step, the, world
de beauvoir: all, blah, not, now, of, oh, own, pedestrian, prisstina, pt
lazify - james brown - funky drummer: do, funky, it, me, single, stuff, the, to, version, you
lazify - killing joke - eighty: 2007, digital, love, me, my, new, no, remaster, the, to
trip-hop: does, like, look, part, soul, stem, the, what, you, your
barricada: azulejo, bienvenidos, blanco, en, frío, hay, negra, negro, no, oveja

基于关键词 'love' 的推荐歌单:
1. u loud
2. lazify - killing joke - eighty
3. sun kil moon - among leaf
4. lazify - james brown - funky drummer
5. trip-hop
6. mount kimble
7. nocturne
8. chicago
9. de beauvoir
10. barricada


In [8]:
print(song_df.head())

  artist                   song                                        link  \
0   ABBA  Ahe's My Kind Of Girl  /a/abba/ahes+my+kind+of+girl_20598417.html   
1   ABBA       Andante, Andante       /a/abba/andante+andante_20002708.html   
2   ABBA         As Good As New        /a/abba/as+good+as+new_20003033.html   
3   ABBA                   Bang                  /a/abba/bang_20598415.html   
4   ABBA       Bang-A-Boomerang      /a/abba/bang+a+boomerang_20002668.html   

                                                text  
0  Look at her face, it's a wonderful face  \r\nA...  
1  Take it easy with me, please  \r\nTouch me gen...  
2  I'll never know why I had to go  \r\nWhy I had...  
3  Making somebody happy is a question of give an...  
4  Making somebody happy is a question of give an...  


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

song_df['text'] = song_df['text'].apply(preprocess_text)

# 提取随机歌单中的歌曲，并创建一个独立的df用于合并trackname和song
random_playlist_songs = user_playlists[user_playlists['playlistname'].isin(random_playlists['playlistname'])]
track_song_df = pd.merge(random_playlist_songs, song_df, left_on='trackname', right_on='song', how='inner')

# 提取每首歌曲的TF-IDF特征
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(track_song_df['text'])

# 提取每个歌单的所有歌词并合并为一个文本
playlist_lyrics = track_song_df.groupby('playlistname')['text'].apply(' '.join).reset_index()

# 提取关键词
def extract_keywords(text, vectorizer, top_n=10):
    X = vectorizer.transform([text])
    indices = X.toarray().argsort()[0, -top_n:][::-1]
    feature_names = vectorizer.get_feature_names_out()
    keywords = [feature_names[i] for i in indices]
    return ' '.join(keywords)

# 提取每个歌单的关键词
playlist_lyrics['keywords'] = playlist_lyrics['text'].apply(lambda x: extract_keywords(x, tfidf_vectorizer))

print("\n每个歌单的关键词:")
print(playlist_lyrics[['playlistname', 'keywords']])

# 合并所有歌单的关键词
all_keywords = ' '.join(playlist_lyrics['keywords'].tolist())

# 使用合并的关键词搜索歌曲
def recommend_songs_based_on_keywords(keywords, song_df, vectorizer, top_n=20):
    keywords_vector = vectorizer.transform([keywords])
    song_vectors = vectorizer.transform(song_df['text'])
    similarities = cosine_similarity(keywords_vector, song_vectors).flatten()
    top_indices = similarities.argsort()[-top_n*2:][::-1]  # 获取多一些，以防去重后不足20首
    recommended_songs = song_df.iloc[top_indices]
    recommended_songs = recommended_songs.drop_duplicates(subset='song').head(top_n)
    return recommended_songs

# 推荐20首基于合并关键词的歌曲
recommended_songs = recommend_songs_based_on_keywords(all_keywords, song_df, tfidf_vectorizer)
print(f"\n基于歌单关键词推荐的20首歌:")
for i, row in recommended_songs.iterrows():
    print(f"{i+1}. {row['song']}")


每个歌单的关键词:
                           playlistname  \
0                               chicago   
1                           de beauvoir   
2  lazify - james brown - funky drummer   
3        lazify - killing joke - eighty   
4                          mount kimble   
5             sun kil moon - among leaf   
6                              trip-hop   
7                                u loud   

                                            keywords  
0          tell wan run na want body oh yo oooh make  
1  blah yeah voice life need bring faithfully lov...  
2  jane mary river water washing know drop im dip...  
3  dance war desire come want reptile tick floor ...  
4       ra slow rida tari fall skip ta body beat let  
5  young billy love strong said girl nail board h...  
6     dont numb wan valentine let na ra love feel im  
7   fever got heaven dont oh thing know im burn kiss  

基于歌单关键词推荐的20首歌:
48523. Slow
54699. Tell Me
8785. Float N' Slow - My Gift To You Bad Boy (Remix)
31657. Ph